# 보험 약관 챗봇 에이전트 (RAG-약관)

In [17]:
# === BLK1: Install & Env Check (revised with auto-discovery) ===
%pip install -q langchain langchain-openai langchain-community langchain-text-splitters \
               faiss-cpu pymupdf tiktoken pydantic "unstructured[pdf]" pytesseract pillow \
               pdfminer.six pypdf python-dotenv

import os, sys, platform, shutil
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

# --- 필수 키 점검 (OpenAI) ---
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise EnvironmentError("❌ OPENAI_API_KEY가 설정되지 않았습니다. .env에 추가 후 다시 실행하세요.")

# --- 파일 경로 자동 탐색 ---
CANDIDATES = [
    Path("/mnt/data/DBIS.pdf"),
    Path("./policies/DBIS.pdf"),
    Path("./DBIS.pdf"),
]

found = None
for p in CANDIDATES:
    if p.exists():
        found = p
        break

# 그래도 못 찾으면 작업 디렉토리 아래에서 재귀 탐색
if not found:
    for root, dirs, files in os.walk(".", topdown=True):
        if "DBIS.pdf" in files:
            found = Path(root) / "DBIS.pdf"
            break

if not found:
    raise FileNotFoundError(
        "❌ DBIS.pdf를 찾지 못했습니다. 다음 위치 중 하나에 파일을 두고 다시 실행하세요:\n"
        " - ./policies/DBIS.pdf\n - ./DBIS.pdf\n - /mnt/data/DBIS.pdf"
    )

# ./policies 로 정규화
POLICY_DIR = Path("./policies")
POLICY_DIR.mkdir(parents=True, exist_ok=True)
POLICY_PATH = POLICY_DIR / "DBIS.pdf"

if found.resolve() != POLICY_PATH.resolve():
    try:
        shutil.copy2(found, POLICY_PATH)
        msg = f"📄 파일을 {found} → {POLICY_PATH} 로 복사했습니다."
    except Exception as e:
        # 복사가 실패해도 원본 경로를 그대로 사용
        POLICY_PATH = found
        msg = f"⚠️ policies로 복사 실패. 원본 경로를 그대로 사용합니다: {POLICY_PATH}\n사유: {e}"
else:
    msg = f"📄 파일이 이미 {POLICY_PATH} 에 있습니다."

print("✅ BLK1 OK | OpenAI Key 확인 완료")
print("OS:", platform.system(), platform.version())
print(msg)
print("사용 파일 경로:", POLICY_PATH.resolve())


Note: you may need to restart the kernel to use updated packages.
✅ BLK1 OK | OpenAI Key 확인 완료
OS: Windows 10.0.19045
📄 파일이 이미 policies\DBIS.pdf 에 있습니다.
사용 파일 경로: C:\Users\배성우\Desktop\TIL\LLM-agent\policies\DBIS.pdf



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# === BLK2-MULTI: Load multiple PDFs (with manual OCR fallback) ===
from pathlib import Path
from io import BytesIO
from collections import defaultdict, Counter
import platform, os

import fitz  # PyMuPDF
import pytesseract
from PIL import Image

try:
    from langchain_core.documents import Document
except Exception:
    from langchain.schema import Document

# 0) (선택) 특정 파일만 제한하고 싶으면 아래 리스트에 파일명을 적으세요.
#    예) INCLUDE_FILES = ["db(2p).pdf", "DBIS.pdf"]
INCLUDE_FILES = None   # 또는 ["db(2p).pdf"]

# 1) Tesseract 경로 자동 설정 (Windows)
if platform.system() == "Windows":
    candidates = [
        os.getenv("TESSERACT_CMD"),
        r"C:\Program Files\Tesseract-OCR\tesseract.exe",
        r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe",
    ]
    for c in candidates:
        if c and Path(c).exists():
            pytesseract.pytesseract.tesseract_cmd = c
            print("🧭 Tesseract 경로:", c)
            break

# 2) MuPDF 텍스트 추출
def extract_text_mupdf(pdf_path: Path):
    out = []
    with fitz.open(str(pdf_path)) as doc:
        for i, page in enumerate(doc):
            txt = page.get_text("text") or ""
            out.append(Document(page_content=txt, metadata={"source": str(pdf_path), "page": i}))
    total_chars = sum(len((d.page_content or "").strip()) for d in out)
    return out, total_chars

# 3) 수동 OCR (페이지→이미지→Tesseract)
def ocr_pdf_with_pytesseract(pdf_path: Path, dpi=300, lang="kor+eng"):
    out = []
    with fitz.open(str(pdf_path)) as doc:
        for i, page in enumerate(doc):
            mat = fitz.Matrix(dpi/72, dpi/72)
            pix = page.get_pixmap(matrix=mat, alpha=False)
            img = Image.open(BytesIO(pix.tobytes("png")))
            text = pytesseract.image_to_string(img, lang=lang)
            out.append(Document(page_content=text, metadata={"source": str(pdf_path), "page": i}))
    total_chars = sum(len((d.page_content or "").strip()) for d in out)
    return out, total_chars

# 4) 여러 PDF 로드 (부족하면 OCR로 자동 폴백)
def load_policies_multi(policy_dir="./policies", include_files=None, min_chars=1000, ocr_lang="kor+eng"):
    policy_dir = Path(policy_dir)
    all_paths = sorted(policy_dir.glob("*.pdf"))
    if include_files:
        name_set = {n.lower() for n in include_files}
        all_paths = [p for p in all_paths if p.name.lower() in name_set]
    if not all_paths:
        raise FileNotFoundError(f"❌ PDF가 없습니다: {policy_dir.resolve()} (필터: {include_files})")

    loaded_docs = []
    audit_rows = []
    for pdf in all_paths:
        mdocs, mchars = extract_text_mupdf(pdf)
        mode = "mupdf"
        if mchars < min_chars:  # 텍스트 너무 적으면 OCR 시도
            print(f"⚠️ {pdf.name}: 텍스트 부족({mchars}) → OCR 시도")
            odocs, ochars = ocr_pdf_with_pytesseract(pdf, dpi=300, lang=ocr_lang)
            if ochars > 0:
                mdocs, mchars, mode = odocs, ochars, "ocr"
            else:
                print(f"  → OCR도 실패(0 chars). 이 파일은 텍스트 인식 불가일 수 있습니다.")
        loaded_docs.extend(mdocs)
        audit_rows.append({"file": pdf.name, "pages": len(mdocs), "chars": mchars, "mode": mode})
    return loaded_docs, audit_rows

# 5) 실행
POLICY_DIR = Path("./policies")
docs, audit = load_policies_multi(POLICY_DIR, include_files=INCLUDE_FILES, min_chars=1000, ocr_lang="kor+eng")

# 6) 결과 요약/프리뷰
print("\n=== Loaded PDFs ===")
for row in audit:
    print(f"- {row['file']}: pages={row['pages']}, chars={row['chars']}, mode={row['mode']}")
print(f"총 문서 청크(페이지) 수: {len(docs)}")

# 프리뷰
def inspect_docs(docs, max_preview=5):
    exts = []
    empty = 0
    for d in docs:
        src = d.metadata.get("source") or ""
        exts.append(Path(src).suffix.lower())
        if not (d.page_content or "").strip():
            empty += 1
    print("빈/짧은 문서 수:", empty)
    print("확장자 분포:", Counter(exts))
    for i, d in enumerate(docs[:max_preview], start=1):
        src = Path(d.metadata.get("source") or "n/a").name
        page = d.metadata.get("page", "n/a")
        txt = (d.page_content or "").strip().replace("\n", " ")
        print(f"[샘플 {i}] {src} (p.{page}) :: {txt[:300]}{'...' if len(txt)>300 else ''}")

inspect_docs(docs)

print("\n✅ BLK2-MULTI OK | 여러 PDF 로딩 완료")
print("   포함 파일(일부):", sorted({Path(d.metadata.get('source','')).name for d in docs})[:10])

In [20]:
# === BLK2-FIX: Set Tesseract path (Windows) + Manual OCR retry ===
import os, platform
from pathlib import Path
from io import BytesIO

import fitz  # PyMuPDF
import pytesseract
from PIL import Image

try:
    from langchain_core.documents import Document
except Exception:
    from langchain.schema import Document

# 0) Tesseract 경로 설정: 환경변수 > 흔한 설치 경로 > 사용자 직접 지정 변수
TESS_ENV = os.getenv("TESSERACT_CMD")
candidates = [TESS_ENV] if TESS_ENV else []
if platform.system() == "Windows":
    candidates += [
        r"C:\Program Files\Tesseract-OCR\tesseract.exe",
        r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe",
    ]
# 사용자가 직접 경로를 알고 있다면 여기 한 줄만 수정해서 넣어도 OK:
# candidates.insert(0, r"C:\Path\To\Your\Tesseract-OCR\tesseract.exe")

found_path = None
for c in candidates:
    if c and Path(c).exists():
        found_path = c
        break

if found_path:
    pytesseract.pytesseract.tesseract_cmd = found_path
    print("✅ Tesseract 경로 설정:", found_path)
else:
    raise FileNotFoundError(
        "❌ Tesseract 실행파일을 찾지 못했습니다.\n"
        " - Windows라면 보통: C:\\Program Files\\Tesseract-OCR\\tesseract.exe\n"
        " - 설치 후 위 경로를 확인하거나, 환경변수 TESSERACT_CMD 에 경로를 지정하세요.\n"
        " - 경로를 알면 위 코드의 candidates.insert(0, ...) 줄에 직접 넣어도 됩니다."
    )

# 1) MuPDF 텍스트 추출
def extract_text_mupdf(pdf_path: Path):
    docs = []
    with fitz.open(str(pdf_path)) as doc:
        for i, page in enumerate(doc):
            txt = page.get_text("text") or ""
            docs.append(Document(page_content=txt, metadata={"source": str(pdf_path), "page": i}))
    total_chars = sum(len((d.page_content or "").strip()) for d in docs)
    return docs, total_chars

# 2) 수동 OCR: 페이지→이미지→Tesseract
def ocr_pdf_with_pytesseract(pdf_path: Path, dpi=300, lang="kor+eng"):
    docs = []
    with fitz.open(str(pdf_path)) as doc:
        for i, page in enumerate(doc):
            mat = fitz.Matrix(dpi/72, dpi/72)
            pix = page.get_pixmap(matrix=mat, alpha=False)
            img = Image.open(BytesIO(pix.tobytes("png")))
            text = pytesseract.image_to_string(img, lang=lang)
            docs.append(Document(page_content=text, metadata={"source": str(pdf_path), "page": i}))
    total_chars = sum(len((d.page_content or "").strip()) for d in docs)
    return docs, total_chars

# 3) 실제 실행 (POLICY_PATH는 1번 블록에서 설정됨)
MIN_CHARS = 1000
mupdf_docs, mupdf_chars = extract_text_mupdf(POLICY_PATH)
print(f"🔎 MuPDF 추출 페이지:{len(mupdf_docs)} | 총문자수:{mupdf_chars}")

if mupdf_chars >= MIN_CHARS:
    docs = mupdf_docs
    load_mode = "mupdf"
else:
    print("⚠️ 텍스트가 부족하여 수동 OCR로 재시도합니다...(kor+eng)")
    ocr_docs, ocr_chars = ocr_pdf_with_pytesseract(POLICY_PATH, dpi=300, lang="kor+eng")
    print(f"🧾 OCR 추출 페이지:{len(ocr_docs)} | 총문자수:{ocr_chars}")
    if ocr_chars == 0:
        raise RuntimeError("OCR 결과가 비었습니다. Tesseract 설치/한글 언어팩(kor) 확인 후 다시 시도하세요.")
    docs = ocr_docs
    load_mode = "ocr"

print(f"✅ 로딩 완료 | 모드:{load_mode} | 문서 수:{len(docs)}")

# 4) 간단 프리뷰
from collections import Counter
def inspect_docs(docs, max_preview=3):
    empty_cnt = sum(1 for d in docs if not (d.page_content or "").strip())
    print("빈/짧은 문서 수:", empty_cnt)
    for i, d in enumerate(docs[:max_preview], start=1):
        src = d.metadata.get("source") or "n/a"
        page = d.metadata.get("page", "n/a")
        txt = (d.page_content or "").strip().replace("\n", " ")
        print(f"[샘플 {i}] {Path(src).name} (p.{page}) :: {txt[:300]}{'...' if len(txt)>300 else ''}")

inspect_docs(docs)

✅ Tesseract 경로 설정: C:\Program Files\Tesseract-OCR\tesseract.exe
🔎 MuPDF 추출 페이지:43 | 총문자수:0
⚠️ 텍스트가 부족하여 수동 OCR로 재시도합니다...(kor+eng)
🧾 OCR 추출 페이지:43 | 총문자수:95988
✅ 로딩 완료 | 모드:ocr | 문서 수:43
빈/짧은 문서 수: 0
[샘플 1] DBIS.pdf (p.0) :: phoBerze SIH OFS Oe IT  BD FLUE eusol  1 ABLOI ALB! SAM, 7UAAME 0] SEQ SQUSAHS AHopo] Qos Hojaz mayors AAA Mt ASAE AALS] SOlSpAl7| BEEPLICH Sse G40] Qe AWS AO] HAA/SO| USA Hoos Fees ol Cyo-oj ABAtS el AALSEAE HSS SO] OS SHO] ASAoa o| SSNS DHSS 2024.12.26 AH = SoAseuct AY ALSFAPAIAL = 2024.12.26 YO] ...
[샘플 2] DBIS.pdf (p.1) :: <r BD or = dlp {J  phe  Ik . = jo ~ A _ IN — or oO — 0 cS = 2 ni = 70 nip io jo!  ZO  FISAH| A} SIL CI}  20 HH ZU HOS  AOE SSH Y241107M)(AAY 2A)  — _  ;  a [o)  HSS 72)  HS Zeo|cpo|z  cI TT  OF  XO  Zl OF Xf  HAS (SIL-O| HZ! - 2025-01-16)  Tj] 2S At HOAs  ri  82,891  <F g ze s ar * .~ tH iO 1 Ei nk ...
[샘플 3] DBIS.pdf (p.2) :: tnt, Fes pe DBAs BHAI ALAEQIAE SOlAPSH  - DAPAME SS 7A A Oe ASS ASA] SAL OrAl7| BPEPL{Ct. -  DFA Sea 7Ede S  © HMOLHS

In [ ]:
# === BLK3: Build FAISS index + compression retriever + synonyms-aided search ===
from typing import List, Dict, Tuple
from pathlib import Path
from collections import OrderedDict

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# ----- 0) 문서 존재 확인 (BLK2에서 생성된 docs 사용) -----
try:
    assert isinstance(docs, list) and len(docs) > 0
except Exception:
    raise RuntimeError("docs 변수가 없습니다. 블록 2까지 성공적으로 실행했는지 확인하세요.")

# ----- 1) 안전한 인덱스 빌드 -----
def build_vectorstore_from_docs(
    documents: List,
    chunk_size: int = 500,
    chunk_overlap: int = 50,
    min_char: int = 10,
):
    assert chunk_size > 0 and 0 <= chunk_overlap < chunk_size
    filtered = [
        d for d in documents
        if isinstance(getattr(d, "page_content", None), str) and len(d.page_content.strip()) >= min_char
    ]
    if not filtered:
        raise ValueError("문서 텍스트가 비어 있습니다. OCR/언어팩/파일 품질을 확인하세요.")

    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = splitter.split_documents(filtered)
    if not split_docs:
        raise ValueError("청크 분할 결과가 0개입니다. chunk_size/overlap 또는 문서 품질을 확인하세요.")

    vs = FAISS.from_documents(split_docs, OpenAIEmbeddings())
    return vs, split_docs

vs, split_docs = build_vectorstore_from_docs(docs, chunk_size=500, chunk_overlap=50)
print(f"✅ 인덱스 빌드 완료 | 청크 수: {len(split_docs)}")

# ----- 2) 컨텍스트 압축 리트리버 (토큰 절약) -----
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # 모델은 요구사항대로 유지
compressor = LLMChainExtractor.from_llm(llm)
base_retriever = vs.as_retriever(search_kwargs={"k": 8})
retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=base_retriever)
print("✅ 리트리버 구성 완료 (k=8 + LLM 압축)")

# ----- 3) 질병/수술 동의어 사전 -----
MAJOR_DISEASE_SYNONYMS: Dict[str, List[str]] = {
    "암": ["암", "악성신생물", "cancer", "malignant neoplasm"],
    "뇌졸중": ["뇌졸중", "뇌졸증", "뇌혈관질환", "중풍", "stroke", "cerebrovascular accident", "CVA"],
    "급성심근경색": ["급성심근경색", "심근경색", "acute myocardial infarction", "AMI", "myocardial infarction"],
}

SURGERY_SYNONYMS: Dict[str, List[str]] = {
    "관상동맥우회술": [
        "관상동맥 우회술", "관상동맥우회로술", "관상동맥우회술",
        "CABG", "Coronary Artery Bypass Grafting", "coronary artery bypass"
    ],
    "관상동맥 스텐트삽입술": [
        "스텐트 삽입술", "관상동맥 스텐트 삽입술", "PCI",
        "percutaneous coronary intervention", "stent"
    ],
    "백내장수술": [
        "백내장 수술", "cataract surgery", "phacoemulsification"
    ],
    "충수절제술": [
        "충수절제술", "맹장 수술", "appendectomy", "appendicectomy"
    ],
    "담낭절제술": [
        "담낭절제술", "담낭 제거", "cholecystectomy"
    ],
    "제왕절개": [
        "제왕절개", "c-section", "cesarean", "cesarean section"
    ],
    "슬관절치환술": [
        "무릎 인공관절 치환술", "슬관절 치환술", "TKA", "total knee arthroplasty"
    ],
    "고관절치환술": [
        "고관절 치환술", "THA", "total hip arthroplasty"
    ],
}

# 병합된 사전(질병+수술)
ALL_SYNONYMS: Dict[str, List[str]] = {**MAJOR_DISEASE_SYNONYMS, **SURGERY_SYNONYMS}

def expand_terms_with_synonyms(query: str) -> List[str]:
    """
    입력 질의에 등장하는 표현을 기준으로 동의어를 확장해 반환.
    - 완전일치/부분일치(공백 제거/소문자) 기준으로 간단 매칭.
    """
    qnorm = query.lower().replace(" ", "")
    expanded = [query]
    for key, syns in ALL_SYNONYMS.items():
        for term in [key] + syns:
            tnorm = term.lower().replace(" ", "")
            if tnorm in qnorm or qnorm in tnorm:
                expanded.extend(syns + [key])
                break
    # 순서 보존 중복 제거
    dedup = list(OrderedDict((s, None) for s in expanded).keys())
    return dedup

# ----- 4) 동의어 확장 RAG 검색 헬퍼 -----
def search_with_synonyms(query: str, top_k: int = 6) -> List[Tuple[str, int, str]]:
    """
    동의어 확장 후 각 질의어로 검색하여 스니펫을 모으고,
    (source, page) 기준으로 중복을 제거해 상위 top_k만 반환.
    Returns: [(source, page, text_snippet), ...]
    """
    terms = expand_terms_with_synonyms(query)
    bucket: Dict[Tuple[str, int], str] = OrderedDict()
    for t in terms:
        try:
            docs = retriever.get_relevant_documents(t)
        except Exception:
            docs = []
        for d in docs:
            src = d.metadata.get("source") or d.metadata.get("file_path") or "source:n/a"
            page = d.metadata.get("page")
            key = (src, page if page is not None else -1)
            if key not in bucket and (d.page_content or "").strip():
                bucket[key] = d.page_content.strip()
            if len(bucket) >= top_k:
                break
        if len(bucket) >= top_k:
            break
    out = []
    for (src, page), txt in list(bucket.items())[:top_k]:
        out.append((Path(src).name, page, txt[:700]))
    return out

# ----- 5) 간단 동작 확인 -----
print("\n🔎 테스트 검색 예시: '관상동맥 우회술 청구'")
_preview = search_with_synonyms("관상동맥 우회술 청구", top_k=3)
for i, (src, page, txt) in enumerate(_preview, start=1):
    print(f"[Hit {i}] {src} (p.{page}) :: {txt[:200]}{'...' if len(txt)>200 else ''}")

print("\n✅ BLK3 OK | 인덱스 & 동의어 검색 준비 완료")


✅ 인덱스 빌드 완료 | 청크 수: 283
✅ 리트리버 구성 완료 (k=8 + LLM 압축)

🔎 테스트 검색 예시: '관상동맥 우회술 청구'


C:\Users\배성우\AppData\Local\Temp\ipykernel_16840\4130679557.py:118: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(t)


[Hit 1] DBIS.pdf (p.6) :: LABA902RP
[Hit 2] DBIS.pdf (p.7) :: LABA902RP
[Hit 3] DBIS.pdf (p.8) :: LABA902RP

✅ BLK3 OK | 인덱스 & 동의어 검색 준비 완료


In [22]:
# === BLK4: Structured outputs (Summary / Grading / Surgery Guide) ===
from typing import List, Dict, Tuple, Optional
from collections import OrderedDict

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# 안전 확인 (앞 블록에서 생성됨)
try:
    retriever, llm
except NameError as e:
    raise RuntimeError("retriever/llm가 없습니다. 블록 3을 먼저 실행하세요.") from e

# --------------------------
# 공통 헬퍼
# --------------------------
AGE_BUCKETS = [(0, 19), (20, 29), (30, 39), (40, 49), (50, 59), (60, 200)]

def build_context_from_queries(queries: List[str], top_k: int = 4, max_snippets: int = 18, max_chars: int = 1000):
    """
    검색어 리스트로 동의어 확장 검색 → (source, page) 기준 중복 제거 → 문맥블록/인용목록 생성
    """
    from pathlib import Path
    bucket: OrderedDict[Tuple[str, int], str] = OrderedDict()
    for q in queries:
        hits = search_with_synonyms(q, top_k=top_k)
        for src, page, txt in hits:
            key = (src, -1 if page is None else int(page))
            if key not in bucket and txt:
                bucket[key] = txt[:max_chars]
            if len(bucket) >= max_snippets:
                break
        if len(bucket) >= max_snippets:
            break

    # 문맥 블록과 인용 리스트
    ctx_lines, citations = [], []
    for i, ((src, page), txt) in enumerate(bucket.items(), start=1):
        ctx_lines.append(f"[Doc {i}] {src} (p.{page})\n{txt}")
        citations.append(f"{src} p.{page}")
    context_block = "\n\n".join(ctx_lines) if ctx_lines else "관련 근거를 찾지 못했습니다."
    return context_block, sorted(set(citations))


# --------------------------
# (A) 약관 요약 ≤5장
# --------------------------
class PolicySummary(BaseModel):
    executive_summary: str = Field(..., description="핵심 요약(10~15문장)")
    coverage_overview: List[str] = Field(..., description="주요 보장 항목(10~20개)")
    exclusions_limitations: List[str] = Field(..., description="주요 면책/제한 사항(5~15개)")
    claim_process: List[str] = Field(..., description="청구 절차 요약(5~10단계)")
    caveats: List[str] = Field(..., description="주의/모호/확인 필요 사항")
    citations: List[str] = Field(..., description="참고 원문(파일명 p.xx)")

summary_parser = PydanticOutputParser(pydantic_object=PolicySummary)
summary_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 보험 약관 전문 요약자다. 과장/추측 금지, 반드시 원문 근거에 의존하라. "
     "결과는 최대 A4 5장 분량에 맞게 간결·비중복으로 요약한다. "
     "이 답변은 법률/의료/재정 자문이 아니며, 최종 판단은 보험사/전문가 상담이 필요함을 전제로 한다."),
    ("human",
     "컨텍스트(약관 발췌):\n{context}\n\n"
     "요구사항:\n- 종합 요약(10~15문장)\n- 보장 항목 목록(10~20)\n- 면책/제한(5~15)\n- 청구 절차(5~10단계)\n- 주의/모호/확인 필요사항\n"
     "- 원문 인용 목록(파일명 p.xx)\n\n{format_instructions}")
]).partial(format_instructions=summary_parser.get_format_instructions())

def summarize_policy_structured() -> dict:
    queries = [
        "보장 내용 요약", "면책", "제외", "청구 절차", "가입/납입/갱신", "해지", "감액", "대기기간",
        "3대 질병", "정의", "특약", "면책 사유", "필요서류"
    ]
    context_block, cites = build_context_from_queries(queries, top_k=4, max_snippets=20, max_chars=1200)
    chain = summary_prompt | llm | summary_parser
    out = chain.invoke({"context": context_block})
    result = out.model_dump()
    result["citations"] = cites
    return result


# --------------------------
# (B) 약관 평가 0~10점
# --------------------------
class ScoreDetail(BaseModel):
    criterion: str
    score: float
    rationale: str
    citations: List[str]

class PolicyGrade(BaseModel):
    overall_score: float
    scale: str
    details: List[ScoreDetail]
    gaps: List[str]
    improvement_suggestions: List[str]
    notes: str

grade_parser = PydanticOutputParser(pydantic_object=PolicyGrade)
grade_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 보험 약관 품질을 0~10점으로 평가한다. "
     "평가축: ①연령대별 보험료 적합성 ②3대 질병 진단비 보장 적정성 ③불필요/중복 약관 여부. "
     "판단은 약관 원문 근거에 기반해야 하며, 근거 부족은 감점 및 '불확실' 표기. "
     "이 결과는 정보 제공 목적이며, 최종 판단은 보험사/전문가 상담이 필요하다."),
    ("human",
     "컨텍스트(약관 발췌):\n{context}\n\n"
     "평가 기준:\n- 연령대 범주: {age_buckets}\n- 3대 질병(암/뇌졸중/급성심근경색)\n- 불필요/중복/모호 조항\n\n"
     "가중치: 동일 가중치(각 1/3)\n\n{format_instructions}")
]).partial(format_instructions=grade_parser.get_format_instructions())

def grade_policy_structured() -> dict:
    price_terms = ["보험료", "월 보험료", "연령", "나이", "보험료표", "납입", "가입연령", "갱신", "비갱신"]
    disease_terms = [
        "암", "악성신생물", "cancer",
        "뇌졸중", "stroke", "CVA",
        "급성심근경색", "AMI", "myocardial infarction"
    ]
    queries = price_terms + disease_terms + ["면책", "제외", "중복", "특약", "모호", "단서"]
    context_block, cites = build_context_from_queries(queries, top_k=3, max_snippets=22, max_chars=900)

    chain = grade_prompt | llm | grade_parser
    out = chain.invoke({"context": context_block, "age_buckets": AGE_BUCKETS})
    result = out.model_dump()
    # 전체 인용 요약을 notes에 병기
    note_tail = f"\n\n참고 근거(집계): {', '.join(cites[:30])}" if cites else ""
    result["notes"] = (result.get("notes") or "") + note_tail
    return result


# --------------------------
# (C) 수술별 보장/청구 안내
# --------------------------
class SurgeryGuide(BaseModel):
    surgery: str
    patient_age: Optional[int]
    matched_benefits: List[str]
    payout_estimates: List[str]
    waiting_or_exclusions: List[str]
    claim_steps: List[str]
    required_documents: List[str]
    caveats: List[str]
    citations: List[str]

surgery_parser = PydanticOutputParser(pydantic_object=SurgeryGuide)
surgery_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 특정 수술에 대해 약관에서 보장/면책/청구조건을 찾아 정리한다. "
     "금액/비율/기한/예외/대상 여부/의학적 정의를 가능하면 원문에 근거해 요약하고, 불확실하면 '불확실' 표기. "
     "이 답변은 정보 제공 목적이며, 최종 청구/적용은 보험사 심사/전문가 상담이 필요하다."),
    ("human",
     "수술명: {surgery}\n나이: {age}\n\n"
     "컨텍스트(약관 발췌):\n{context}\n\n"
     "요구사항:\n- 관련 보장 항목\n- 지급액/지급비율 추정(근거 포함)\n- 대기/면책/예외 조건\n- 청구 절차 단계\n- 필요서류\n- 주의/모호 사항\n\n"
     "{format_instructions}")
]).partial(format_instructions=surgery_parser.get_format_instructions())

def surgery_coverage_structured(surgery: str, age: Optional[int] = None) -> dict:
    # 동의어 확장 기반 쿼리 생성
    terms = expand_terms_with_synonyms(surgery)
    base_queries = []
    for t in terms:
        base_queries += [t, f"{t} 보장", f"{t} 수술비", f"{t} 진단비", f"{t} 청구", f"{t} 면책", f"{t} 대기기간", f"{t} 필요서류", f"{t} 정의"]
    context_block, cites = build_context_from_queries(base_queries, top_k=4, max_snippets=20, max_chars=900)

    chain = surgery_prompt | llm | surgery_parser
    out = chain.invoke({"surgery": surgery, "age": age, "context": context_block})
    result = out.model_dump()
    result["citations"] = cites
    return result


print("✅ BLK4 OK | 구조화 함수 3종 준비 완료 (summarize_policy_structured / grade_policy_structured / surgery_coverage_structured)")


✅ BLK4 OK | 구조화 함수 3종 준비 완료 (summarize_policy_structured / grade_policy_structured / surgery_coverage_structured)


In [23]:
# === BLK5: Wrap tools + Create Agent ===
from typing import Optional, List
from langchain_core.tools import tool
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

# 안전 확인 (앞 블록에서 생성됨)
try:
    llm, retriever
    summarize_policy_structured
    grade_policy_structured
    surgery_coverage_structured
    search_with_synonyms
except NameError as e:
    raise RuntimeError("필수 객체/함수가 없습니다. 블록 3~4를 먼저 실행하세요.") from e

# --------------------------
# 1) 툴 래핑
# --------------------------
@tool("summarize_policy", return_direct=False)
def summarize_policy() -> dict:
    """
    업로드된 약관(DBIS.pdf)을 최대 5장 분량으로 구조화 요약합니다.
    """
    return summarize_policy_structured()

@tool("grade_policy", return_direct=False)
def grade_policy() -> dict:
    """
    약관을 0~10점으로 평가하고, 세부 근거와 개선 제안을 제공합니다.
    평가축: 연령대별 보험료 적합성 / 3대 질병 진단비 적정성 / 불필요·중복 조항 여부
    """
    return grade_policy_structured()

@tool("surgery_coverage", return_direct=False)
def surgery_coverage(surgery: str, age: Optional[int] = None) -> dict:
    """
    특정 수술에 대해 보장/면책/청구 절차/필요서류를 약관 근거와 함께 구조화해 반환합니다.
    """
    return surgery_coverage_structured(surgery=surgery, age=age)

@tool("knowledge_retriever", return_direct=False)
def knowledge_retriever(query: str, top_k: int = 6) -> str:
    """
    동의어 확장 검색으로 관련 조항 스니펫을 반환합니다. (파일명/페이지 포함)
    """
    hits = search_with_synonyms(query, top_k=top_k)
    if not hits:
        return "관련 근거를 찾지 못했습니다."
    lines = []
    for i, (src, page, txt) in enumerate(hits, start=1):
        lines.append(f"[Hit {i}] {src} (p.{page})\n{txt}")
    return "\n\n".join(lines)

tools = [summarize_policy, grade_policy, surgery_coverage, knowledge_retriever]

# --------------------------
# 2) 에이전트 프롬프트 & 메모리
# --------------------------
agent_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 한국어를 기본으로 답하는 보험 약관 분석 에이전트다. "
     "반드시 약관 원문(파일/페이지)에 근거해 답하고, 불확실하면 '불확실'이라고 명시한다. "
     "법률/의료/재정 자문이 아닌 정보 제공 목적임을 분명히 하고, 최종 판단과 청구는 보험사 심사/전문가 상담이 필요하다고 고지해라. "
     "사용자의 질문이 비 trivial 하거나 근거가 애매하면 다음 툴을 적극 사용하라: "
     "knowledge_retriever(조항 스니펫) / summarize_policy(요약) / grade_policy(평가) / surgery_coverage(수술별 안내)."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# --------------------------
# 3) 에이전트 생성
# --------------------------
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

print("✅ BLK5 OK | 에이전트 준비 완료")
print("   - Tools :", [t.name for t in tools])
print("   - Memory:", type(memory).__name__, "| chat_history size =", len(memory.chat_memory.messages))


✅ BLK5 OK | 에이전트 준비 완료
   - Tools : ['summarize_policy', 'grade_policy', 'surgery_coverage', 'knowledge_retriever']
   - Memory: ConversationBufferMemory | chat_history size = 0


In [24]:
# === BLK6: Run examples ===

# 6-1) 약관 요약 (≤5장 분량)
print("====== [Test 1] summarize_policy ======")
resp_summary = agent_executor.invoke({
    "input": "약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해."
})
resp_summary  # dict 내 result에 구조화 요약이 들어옵니다.


====== [Test 1] summarize_policy ======


> Entering new AgentExecutor chain...

Invoking: `summarize_policy` with `{}`


{'executive_summary': '본 보험약관은 다양한 호흡기 및 알레르기 관련 질환에 대한 보장을 제공하며, 특정 질환에 대한 치료비와 약제비를 보장한다. 보장 항목에는 천식, 만성폐쇄성폐질환, 알레르기 비염, 기관지염, 폐렴, 기관지 확장증, 호흡기 알레르기 검사, 약제비, 입원 및 통원 치료, 응급 치료, 산소치료, 흡입기 및 보조기구 지원 등이 포함된다. 일부 만성질환 또는 특정 조건 하에 제한이 있으며, 면책 조항으로는 자가진단, 미승인 약물 사용, 비보험 대상 치료, 예방적 치료, 비급여 항목, 보험금 청구 후 허위 또는 과장된 진술, 보험사 승인 없이 진행된 치료 등이 있다. 청구 절차는 진단서 제출 → 치료비 영수증 확보 → 보험사에 청구서 제출 → 보험사 검토 → 지급 결정 → 보험금 지급의 순서로 진행된다. 주의사항으로는 질환의 정확한 진단과 치료내역의 명확한 증빙 필요, 비보험 대상 항목의 확인, 치료 시 보험약관 내 보장 범위 재확인, 미승인 약물 또는 치료법 사용 시 보장 제외, 보험금 청구 시 허위 진술 금지, 보험약관 변경 시 최신 내용 확인이 요구된다. 참고 원문은 파일명과 페이지 번호를 기재하였다.', 'coverage_overview': ['천식 및 만성폐쇄성폐질환 치료비 지원', '알레르기 비염 및 기관지염 치료비 보장', '폐렴 및 기관지 확장증 치료비 지원', '호흡기 알레르기 검사 및 진단비 지급', '약제비 및 처방약 지원', '입원 및 통원 치료비 보장', '응급 치료 및 산소치료 지원', '흡입기 및 보조기구 지원', '진단 검사 및 치료 관련 검사비 보장', '응급실 방문 및 치료비 지원'], 'exclusions_limitations': ['자가진단 또는 자가치료 관련 비용 제외', '미승인

{'input': '약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.',
 'chat_history': [HumanMessage(content='약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관 전체의 핵심 내용을 요약하면 다음과 같습니다. 본 보험약관은 호흡기 및 알레르기 관련 질환에 대한 치료비와 약제비를 보장하며, 천식, 만성폐쇄성폐질환, 알레르기 비염, 기관지염, 폐렴, 기관지 확장증, 호흡기 알레르기 검사, 약제비, 입원 및 통원 치료, 응급 치료, 산소치료, 흡입기 및 보조기구 지원 등을 포함합니다. 다만, 자가진단, 미승인 약물 사용, 비보험 대상 치료, 예방적 치료, 비급여 항목, 허위 진술 등은 보장 제외 대상입니다. 보험금 청구는 진단서와 치료비 영수증 제출 후 보험사 검토 과정을 거치며, 정확한 증빙과 사전 확인이 필요합니다. 자세한 내용은 원문(파일명: DBIS.pdf, 페이지: 6~41)을 참고하시기 바랍니다. 이 정보는 참고용임을 유념하시고, 최종 판단과 청구는 보험사 심사 또는 전문가 상담이 필요합니다.', additional_kwargs={}, response_metadata={})],
 'output': '약관 전체의 핵심 내용을 요약하면 다음과 같습니다. 본 보험약관은 호흡기 및 알레르기 관련 질환에 대한 치료비와 약제비를 보장하며, 천식, 만성폐쇄성폐질환, 알레르기 비염, 기관지염, 폐렴, 기관지 확장증, 호흡기 알레르기 검사, 약제비, 입원 및 통원 치료, 응급 치료, 산소치료, 흡입기 및 보조기구 지원 등을 포함합니다. 다만, 자가진단, 미승인 약물 사용, 비보험 대상 치료, 예방적 치료, 비급여 항목, 허위 진술 등은 보장 제외 대상입니다. 보험금 청구는 진단서와 치료비 영수증 제출 후 보험사 검토 과정을 거치며, 정확한 증빙과 사전 확

In [ ]:
print(resp_summary)

In [25]:
# 6-2) 약관 품질 평가 (0~10점)
print("====== [Test 2] grade_policy ======")
resp_grade = agent_executor.invoke({
    "input": (
        "이 약관이 전체적으로 잘 되어있는지 0~10점으로 평가해줘. "
        "평가축은 연령대별 보험료 적합성/3대 질병 진단비/불필요·중복 조항이고, "
        "grade_policy 툴을 사용해."
    )
})
resp_grade


====== [Test 2] grade_policy ======


> Entering new AgentExecutor chain...

Invoking: `grade_policy` with `{}`


{'overall_score': 1.0, 'scale': '0~10', 'details': [{'criterion': '연령대별 보험료 적합성', 'score': 0.0, 'rationale': '약관 원문에 연령대별 보험료 산출 기준이나 적합성에 대한 구체적 내용이 확인되지 않음. 문서 내용이 무의미한 문자와 코드로 구성되어 있어 평가 불가.', 'citations': ['Doc 1-10']}, {'criterion': '3대 질병 진단비 보장 적정성', 'score': 0.0, 'rationale': '3대 질병(암, 뇌졸중, 급성심근경색) 관련 보장 내용이 명확히 제시되지 않으며, 관련 조항이 확인되지 않음. 문서 내용이 무의미한 문자와 코드로 구성되어 있어 평가 불가.', 'citations': ['Doc 1-10']}, {'criterion': '불필요/중복 약관 여부', 'score': 1.0, 'rationale': '약관 내용이 무의미한 문자와 코드로 구성되어 있어 중복 또는 불필요 조항 여부 판단 불가. 문서 내용이 명확하지 않음.', 'citations': ['Doc 1-10']}], 'gaps': ['약관 내용이 무의미한 문자와 코드로 구성되어 있어 평가 근거 부족'], 'improvement_suggestions': ['약관 원문을 명확하고 구체적인 내용으로 재작성 필요'], 'notes': '약관 내용이 비정상적이거나 손상된 것으로 보여 평가가 어려움.\n\n참고 근거(집계): DBIS.pdf p.0, DBIS.pdf p.22, DBIS.pdf p.23, DBIS.pdf p.29, DBIS.pdf p.38, DBIS.pdf p.41, DBIS.pdf p.6, DBIS.pdf p.7, DBIS.pdf p.8, DBIS.pdf p.9'}약관

{'input': '이 약관이 전체적으로 잘 되어있는지 0~10점으로 평가해줘. 평가축은 연령대별 보험료 적합성/3대 질병 진단비/불필요·중복 조항이고, grade_policy 툴을 사용해.',
 'chat_history': [HumanMessage(content='약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관 전체의 핵심 내용을 요약하면 다음과 같습니다. 본 보험약관은 호흡기 및 알레르기 관련 질환에 대한 치료비와 약제비를 보장하며, 천식, 만성폐쇄성폐질환, 알레르기 비염, 기관지염, 폐렴, 기관지 확장증, 호흡기 알레르기 검사, 약제비, 입원 및 통원 치료, 응급 치료, 산소치료, 흡입기 및 보조기구 지원 등을 포함합니다. 다만, 자가진단, 미승인 약물 사용, 비보험 대상 치료, 예방적 치료, 비급여 항목, 허위 진술 등은 보장 제외 대상입니다. 보험금 청구는 진단서와 치료비 영수증 제출 후 보험사 검토 과정을 거치며, 정확한 증빙과 사전 확인이 필요합니다. 자세한 내용은 원문(파일명: DBIS.pdf, 페이지: 6~41)을 참고하시기 바랍니다. 이 정보는 참고용임을 유념하시고, 최종 판단과 청구는 보험사 심사 또는 전문가 상담이 필요합니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='이 약관이 전체적으로 잘 되어있는지 0~10점으로 평가해줘. 평가축은 연령대별 보험료 적합성/3대 질병 진단비/불필요·중복 조항이고, grade_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관의 전반적인 평가 점수는 1.0점으로 매우 낮게 나왔으며, 이는 약관 내용이 무의미한 문자와 코드로 구성되어 있어 평가가 어렵기 때문입니다. 

In [26]:
# 6-3) 수술별 보장/청구 안내 (동의어 매칭 예시 1: 한글명)
print("====== [Test 3] surgery_coverage — 관상동맥 우회술 (45세) ======")
resp_surg1 = agent_executor.invoke({
    "input": "surgery_coverage 툴로 '관상동맥 우회술'을 받는다면(나이 45) 보장/면책/청구 단계/필요서류를 근거와 함께 정리해줘."
})
resp_surg1


====== [Test 3] surgery_coverage — 관상동맥 우회술 (45세) ======


> Entering new AgentExecutor chain...

Invoking: `surgery_coverage` with `{'surgery': '관상동맥 우회술', 'age': 45}`


{'surgery': '관상동맥 우회술', 'patient_age': 45, 'matched_benefits': ['관상동맥 우회술에 대한 수술비 보장'], 'payout_estimates': ['보장금액 또는 비율은 약관에 명시되지 않음 (불확실)'], 'waiting_or_exclusions': ['수술이 보험약관상 인정되는 적응증에 해당해야 함', '수술이 비급여 또는 비인정 수술인 경우 제외 가능성'], 'claim_steps': ['진단서 및 수술 관련 서류 준비', '보험사에 청구서 제출'], 'required_documents': ['진단서', '수술확인서', '수술비 영수증'], 'caveats': ['수술이 보험약관상 보장 대상인지 여부 확인 필요', '면책 조항 또는 특정 조건 충족 필요 가능성'], 'citations': ['DBIS.pdf p.10', 'DBIS.pdf p.16', 'DBIS.pdf p.37', 'DBIS.pdf p.39', 'DBIS.pdf p.6', 'DBIS.pdf p.7', 'DBIS.pdf p.8', 'DBIS.pdf p.9']}'관상동맥 우회술'을 받는 경우(나이 45세) 보험 적용은 다음과 같이 정리됩니다.

**보장 단계:**  
- 수술비 보장 가능 (약관상 인정되는 경우)  
- 보장금액 또는 비율은 약관에 명시되어 있지 않으므로 불확실함

**면책 사항:**  
- 수술이 보험약관상 인정되는 적응증에 해당해야 함  
- 비급여 또는 비인정 수술인 경우 제외될 수 있음

**청구 단계:**  
1. 진단서 및 수술 관련 서류 준비  
2. 보험사에 청구서 제출

**필요서류:**  
- 진단서  
- 수술

{'input': "surgery_coverage 툴로 '관상동맥 우회술'을 받는다면(나이 45) 보장/면책/청구 단계/필요서류를 근거와 함께 정리해줘.",
 'chat_history': [HumanMessage(content='약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관 전체의 핵심 내용을 요약하면 다음과 같습니다. 본 보험약관은 호흡기 및 알레르기 관련 질환에 대한 치료비와 약제비를 보장하며, 천식, 만성폐쇄성폐질환, 알레르기 비염, 기관지염, 폐렴, 기관지 확장증, 호흡기 알레르기 검사, 약제비, 입원 및 통원 치료, 응급 치료, 산소치료, 흡입기 및 보조기구 지원 등을 포함합니다. 다만, 자가진단, 미승인 약물 사용, 비보험 대상 치료, 예방적 치료, 비급여 항목, 허위 진술 등은 보장 제외 대상입니다. 보험금 청구는 진단서와 치료비 영수증 제출 후 보험사 검토 과정을 거치며, 정확한 증빙과 사전 확인이 필요합니다. 자세한 내용은 원문(파일명: DBIS.pdf, 페이지: 6~41)을 참고하시기 바랍니다. 이 정보는 참고용임을 유념하시고, 최종 판단과 청구는 보험사 심사 또는 전문가 상담이 필요합니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='이 약관이 전체적으로 잘 되어있는지 0~10점으로 평가해줘. 평가축은 연령대별 보험료 적합성/3대 질병 진단비/불필요·중복 조항이고, grade_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관의 전반적인 평가 점수는 1.0점으로 매우 낮게 나왔으며, 이는 약관 내용이 무의미한 문자와 코드로 구성되어 있어 평가가 어렵기 때문입니다. 연령대별 보험료 적합성, 3대 질병 진

In [27]:
# 6-4) 수술별 보장/청구 안내 (동의어 매칭 예시 2: 영문 약어 CABG)
print("====== [Test 4] surgery_coverage — CABG (관상동맥우회술 동의어) ======")
resp_surg2 = agent_executor.invoke({
    "input": "surgery_coverage 툴로 'CABG' 수술을 받는다면(나이 45) 해당 보장/청구 조건을 알려줘."
})
resp_surg2


====== [Test 4] surgery_coverage — CABG (관상동맥우회술 동의어) ======


> Entering new AgentExecutor chain...

Invoking: `surgery_coverage` with `{'surgery': '관상동맥 우회술', 'age': 45}`


{'surgery': '관상동맥 우회술', 'patient_age': 45, 'matched_benefits': ['관상동맥 우회술에 대한 수술비 보장'], 'payout_estimates': ['보장금액 또는 비율은 약관에 명시되지 않음 (불확실)'], 'waiting_or_exclusions': ['특정 질환 또는 조건에 따라 면책 또는 제한이 있을 수 있음 (약관 상세 내용 필요)'], 'claim_steps': ['진단서 및 수술 관련 서류 준비', '보험사에 청구서 제출'], 'required_documents': ['진단서', '수술확인서', '수술비 영수증'], 'caveats': ['수술이 보험약관에 명시된 조건을 충족해야 함', '면책 조항 또는 특정 예외 사항 존재 가능성'], 'citations': ['DBIS.pdf p.10', 'DBIS.pdf p.13', 'DBIS.pdf p.16', 'DBIS.pdf p.37', 'DBIS.pdf p.39', 'DBIS.pdf p.6', 'DBIS.pdf p.7', 'DBIS.pdf p.8', 'DBIS.pdf p.9']}'관상동맥 우회술(CABG)' 수술을 45세에 받는 경우, 보험 약관에 따라 다음과 같이 보장 및 청구 조건이 정리됩니다.

**보장 내용:**  
- 관상동맥 우회술에 대한 수술비 보장이 가능하며, 구체적인 보장금액 또는 비율은 약관에 명시되어 있지 않으므로 보험사 검토 후 확정됩니다.

**청구 조건:**  
1. 진단서와 수술 관련 서류를 준비해야 합니다.  
2. 보험사에 청구서를 제출하는 절차를 거칩니다.

**필요서류:**  
- 진단서  
- 수술확인서  
- 수

{'input': "surgery_coverage 툴로 'CABG' 수술을 받는다면(나이 45) 해당 보장/청구 조건을 알려줘.",
 'chat_history': [HumanMessage(content='약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관 전체의 핵심 내용을 요약하면 다음과 같습니다. 본 보험약관은 호흡기 및 알레르기 관련 질환에 대한 치료비와 약제비를 보장하며, 천식, 만성폐쇄성폐질환, 알레르기 비염, 기관지염, 폐렴, 기관지 확장증, 호흡기 알레르기 검사, 약제비, 입원 및 통원 치료, 응급 치료, 산소치료, 흡입기 및 보조기구 지원 등을 포함합니다. 다만, 자가진단, 미승인 약물 사용, 비보험 대상 치료, 예방적 치료, 비급여 항목, 허위 진술 등은 보장 제외 대상입니다. 보험금 청구는 진단서와 치료비 영수증 제출 후 보험사 검토 과정을 거치며, 정확한 증빙과 사전 확인이 필요합니다. 자세한 내용은 원문(파일명: DBIS.pdf, 페이지: 6~41)을 참고하시기 바랍니다. 이 정보는 참고용임을 유념하시고, 최종 판단과 청구는 보험사 심사 또는 전문가 상담이 필요합니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='이 약관이 전체적으로 잘 되어있는지 0~10점으로 평가해줘. 평가축은 연령대별 보험료 적합성/3대 질병 진단비/불필요·중복 조항이고, grade_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관의 전반적인 평가 점수는 1.0점으로 매우 낮게 나왔으며, 이는 약관 내용이 무의미한 문자와 코드로 구성되어 있어 평가가 어렵기 때문입니다. 연령대별 보험료 적합성, 3대 질병 진단비 보장 적정성, 불필요

In [28]:
# 6-5) 특정 조항 스니펫 찾기 (예: 3대 질병 중 '급성심근경색' 진단비)
print("====== [Test 5] knowledge_retriever — 급성심근경색 진단비 ======")
resp_hits = agent_executor.invoke({
    "input": "knowledge_retriever 툴로 '급성심근경색 진단비' 관련 조항과 면책/예외를 찾아서 파일명/페이지와 함께 보여줘."
})
resp_hits


====== [Test 5] knowledge_retriever — 급성심근경색 진단비 ======


> Entering new AgentExecutor chain...

Invoking: `knowledge_retriever` with `{'query': '급성심근경색 진단비'}`


[Hit 1] DBIS.pdf (p.6)
LABA902RP

[Hit 2] DBIS.pdf (p.7)
LABA902RP

[Hit 3] DBIS.pdf (p.9)
LABA902RP

[Hit 4] DBIS.pdf (p.0)
ETStat WE AAA AZ Ol 010-5690-9642 SHO|AI www.idbins.com

[Hit 5] DBIS.pdf (p.41)
ZAI Ol Sol B_H]2024-7273 3 (2024.10.31~2025.10.30)약관에서 '급성심근경색 진단비'와 관련된 조항 및 면책/예외 사항을 찾기 위해 검토한 결과, 구체적인 내용은 약관 원문(파일명: DBIS.pdf, 페이지: 6, 7, 9, 41)에서 확인할 수 있습니다. 다만, 상세 내용은 해당 페이지를 참고하셔야 하며, 일반적으로 급성심근경색 진단비는 특정 진단 기준에 따라 지급되며, 다음과 같은 면책/예외가 있을 수 있습니다.

- **면책/예외 사항:**  
  - 자가진단 또는 비승인 치료로 인한 경우 제외  
  - 비보험 대상 또는 보험약관상 인정되지 않는 경우 제외  
  - 특정 기간 내 또는 특정 조건 미충족 시 지급 제외 가능

이 내용은 약관 원문을 참고하였으며, 구체적인 조건과 면책 사항은 보험사 검토 후 확정됩니다. 최종 판단과 청구는 보험사 심사 또는 전문가 상담이 필요함을 유념하시기 바랍니다.

> Finished chain.


{'input': "knowledge_retriever 툴로 '급성심근경색 진단비' 관련 조항과 면책/예외를 찾아서 파일명/페이지와 함께 보여줘.",
 'chat_history': [HumanMessage(content='약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관 전체의 핵심 내용을 요약하면 다음과 같습니다. 본 보험약관은 호흡기 및 알레르기 관련 질환에 대한 치료비와 약제비를 보장하며, 천식, 만성폐쇄성폐질환, 알레르기 비염, 기관지염, 폐렴, 기관지 확장증, 호흡기 알레르기 검사, 약제비, 입원 및 통원 치료, 응급 치료, 산소치료, 흡입기 및 보조기구 지원 등을 포함합니다. 다만, 자가진단, 미승인 약물 사용, 비보험 대상 치료, 예방적 치료, 비급여 항목, 허위 진술 등은 보장 제외 대상입니다. 보험금 청구는 진단서와 치료비 영수증 제출 후 보험사 검토 과정을 거치며, 정확한 증빙과 사전 확인이 필요합니다. 자세한 내용은 원문(파일명: DBIS.pdf, 페이지: 6~41)을 참고하시기 바랍니다. 이 정보는 참고용임을 유념하시고, 최종 판단과 청구는 보험사 심사 또는 전문가 상담이 필요합니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='이 약관이 전체적으로 잘 되어있는지 0~10점으로 평가해줘. 평가축은 연령대별 보험료 적합성/3대 질병 진단비/불필요·중복 조항이고, grade_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관의 전반적인 평가 점수는 1.0점으로 매우 낮게 나왔으며, 이는 약관 내용이 무의미한 문자와 코드로 구성되어 있어 평가가 어렵기 때문입니다. 연령대별 보험료 적합성, 3대 질병 진단비 보

In [29]:
# 6-6) 자유 질문 (에이전트가 툴 선택)
print("====== [Test 6] free-form ======")
resp_free = agent_executor.invoke({
    "input": (
        "내가 앞으로 '백내장수술'을 받을 수도 있어. "
        "해당 수술에 대해 실제 청구 시 주의해야 할 조건이나 대기기간, "
        "필요서류를 약관 근거와 함께 정리해줘."
    )
})
resp_free


====== [Test 6] free-form ======


> Entering new AgentExecutor chain...

Invoking: `surgery_coverage` with `{'surgery': '백내장수술'}`


{'surgery': '백내장수술', 'patient_age': None, 'matched_benefits': ['백내장수술에 대한 보장'], 'payout_estimates': ['보장금액 또는 비율은 명확히 제시되지 않음 (근거: 약관 내 구체적 금액/비율 언급 없음)'], 'waiting_or_exclusions': ['특정 조건 미충족 시 면책 또는 제외 가능성 (약관 내 구체적 내용 불확실)'], 'claim_steps': ['진료 후 보험사에 청구서 제출', '필요서류 제출 (진료기록, 영수증 등)'], 'required_documents': ['진료기록 사본', '영수증 또는 비용 증빙서류', '보험금 청구서'], 'caveats': ['수술이 특정 조건 또는 기준에 부합해야 함 (약관 내 구체적 정의 불확실)', '면책 조항 또는 예외 사항 존재 가능성'], 'citations': ['DBIS.pdf p.10', 'DBIS.pdf p.37', 'DBIS.pdf p.6', 'DBIS.pdf p.7', 'DBIS.pdf p.8', 'DBIS.pdf p.9']}백내장수술에 대해 보험 청구 시 주의해야 할 조건, 대기기간, 필요서류는 다음과 같습니다.

**주의 조건 및 대기기간:**  
- 수술이 보험약관상 인정되는 적응증에 해당해야 하며, 특정 조건 미충족 시 면책 또는 제외될 수 있습니다.  
- 약관 내 구체적인 대기기간 또는 제한 조건은 명확히 제시되지 않았으나, 일반적으로 수술 후 일정 기간 내 또는 특정 조건 미충족 시 보험금 지급이 제한될 수 있습니다.

**필요서류:**  
- 진료기록 사본  
- 영수증 또는 비용 증빙서류  
- 보험금 청구서

이 내용은 약관 원문(파일명: DBIS.pdf, 페이지:

{'input': "내가 앞으로 '백내장수술'을 받을 수도 있어. 해당 수술에 대해 실제 청구 시 주의해야 할 조건이나 대기기간, 필요서류를 약관 근거와 함께 정리해줘.",
 'chat_history': [HumanMessage(content='약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관 전체의 핵심 내용을 요약하면 다음과 같습니다. 본 보험약관은 호흡기 및 알레르기 관련 질환에 대한 치료비와 약제비를 보장하며, 천식, 만성폐쇄성폐질환, 알레르기 비염, 기관지염, 폐렴, 기관지 확장증, 호흡기 알레르기 검사, 약제비, 입원 및 통원 치료, 응급 치료, 산소치료, 흡입기 및 보조기구 지원 등을 포함합니다. 다만, 자가진단, 미승인 약물 사용, 비보험 대상 치료, 예방적 치료, 비급여 항목, 허위 진술 등은 보장 제외 대상입니다. 보험금 청구는 진단서와 치료비 영수증 제출 후 보험사 검토 과정을 거치며, 정확한 증빙과 사전 확인이 필요합니다. 자세한 내용은 원문(파일명: DBIS.pdf, 페이지: 6~41)을 참고하시기 바랍니다. 이 정보는 참고용임을 유념하시고, 최종 판단과 청구는 보험사 심사 또는 전문가 상담이 필요합니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='이 약관이 전체적으로 잘 되어있는지 0~10점으로 평가해줘. 평가축은 연령대별 보험료 적합성/3대 질병 진단비/불필요·중복 조항이고, grade_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관의 전반적인 평가 점수는 1.0점으로 매우 낮게 나왔으며, 이는 약관 내용이 무의미한 문자와 코드로 구성되어 있어 평가가 어렵기 때문입니다. 연령대별 보험료 적합성

In [30]:
# === BLK7: Markdown formatters + save helpers ===
from pathlib import Path
from textwrap import indent
from IPython.display import Markdown, display

# -------- 공통 유틸 --------
def _h(level:int, text:str) -> str:
    return f"{'#'*level} {text.strip()}"

def _bullets(items, level=0) -> str:
    if not items: 
        return ""
    pad = "  " * level
    return "\n".join(f"{pad}- {str(x).strip()}" for x in items)

def _save_md(md_text:str, path:str) -> str:
    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    # Windows 메모장 호환을 위해 utf-8-sig 권장
    p.write_text(md_text, encoding="utf-8-sig")
    return str(p.resolve())

def _show(md_text:str):
    display(Markdown(md_text))

# -------- (A) 약관 요약 렌더러 --------
def render_summary_md(data:dict, title:str="보험약관 핵심 요약") -> str:
    ex = data.get("executive_summary", "")
    cov = data.get("coverage_overview", [])
    excl = data.get("exclusions_limitations", [])
    claim = data.get("claim_process", [])
    cave = data.get("caveats", [])
    cites = data.get("citations", [])

    parts = []
    parts.append(_h(1, title))
    parts.append(_h(2, "1) Executive Summary"))
    parts.append(ex.strip() if isinstance(ex, str) else str(ex))

    parts.append(_h(2, "2) 주요 보장 항목"))
    parts.append(_bullets(cov))

    parts.append(_h(2, "3) 면책/제한 사항"))
    parts.append(_bullets(excl))

    parts.append(_h(2, "4) 청구 절차 (요약)"))
    parts.append(_bullets(claim))

    parts.append(_h(2, "5) 주의/모호/확인 필요 사항"))
    parts.append(_bullets(cave))

    parts.append(_h(3, "참고 근거(파일 p.xx)"))
    parts.append(_bullets(cites))

    return "\n\n".join(parts).strip() + "\n"

def print_summary_md(data:dict, title="보험약관 핵심 요약"):
    md = render_summary_md(data, title=title)
    _show(md)
    return md

def save_summary_md(data:dict, path:str="./export/summary.md", title="보험약관 핵심 요약") -> str:
    md = render_summary_md(data, title=title)
    return _save_md(md, path)

# -------- (B) 약관 평가 렌더러 --------
def render_grade_md(data:dict, title:str="보험약관 품질 평가 (0~10점)") -> str:
    overall = data.get("overall_score", "")
    scale = data.get("scale", "")
    details = data.get("details", [])  # list of {criterion, score, rationale, citations}
    gaps = data.get("gaps", [])
    suggestions = data.get("improvement_suggestions", [])
    notes = data.get("notes", "")

    parts = []
    parts.append(_h(1, title))
    parts.append(f"**종합 점수:** {overall} / 10")
    if scale:
        parts.append(f"**척도/기준:** {scale}")

    parts.append(_h(2, "세부 평가"))
    if details:
        for i, d in enumerate(details, start=1):
            crit = d.get("criterion", "")
            sc = d.get("score", "")
            rat = d.get("rationale", "")
            cit = d.get("citations", [])
            parts.append(_h(3, f"{i}. {crit} — {sc}점"))
            parts.append(rat.strip())
            if cit:
                parts.append(_bullets([f"{c}" for c in cit]))
    else:
        parts.append("_세부 평가 항목이 없습니다._")

    parts.append(_h(2, "확인된 공백/리스크"))
    parts.append(_bullets(gaps) or "_(없음)_")

    parts.append(_h(2, "개선 제안"))
    parts.append(_bullets(suggestions) or "_(없음)_")

    if notes:
        parts.append(_h(3, "비고"))
        parts.append(notes.strip())

    return "\n\n".join(parts).strip() + "\n"

def print_grade_md(data:dict, title="보험약관 품질 평가 (0~10점)"):
    md = render_grade_md(data, title=title)
    _show(md)
    return md

def save_grade_md(data:dict, path:str="./export/grade.md", title="보험약관 품질 평가 (0~10점)") -> str:
    md = render_grade_md(data, title=title)
    return _save_md(md, path)

# -------- (C) 수술/질병 안내 렌더러 --------
def render_surgery_md(data:dict, title_prefix:str="수술·질병 보장/청구 안내") -> str:
    surgery = data.get("surgery", "")
    age = data.get("patient_age", None)
    benefits = data.get("matched_benefits", [])
    payouts = data.get("payout_estimates", [])
    waits = data.get("waiting_or_exclusions", [])
    steps = data.get("claim_steps", [])
    reqs = data.get("required_documents", [])
    cave = data.get("caveats", [])
    cites = data.get("citations", [])

    title = f"{title_prefix} — {surgery}" if surgery else title_prefix
    parts = []
    parts.append(_h(1, title))
    if age is not None:
        parts.append(f"**환자 나이:** {age}세")

    parts.append(_h(2, "관련 보장 항목"))
    parts.append(_bullets(benefits) or "_(해당 항목 없음)_")

    parts.append(_h(2, "지급액/지급비율 (근거 포함)"))
    parts.append(_bullets(payouts) or "_(명시된 지급 기준 없음)_")

    parts.append(_h(2, "대기기간/면책/예외"))
    parts.append(_bullets(waits) or "_(명시된 조건 없음)_")

    parts.append(_h(2, "청구 절차"))
    parts.append(_bullets(steps) or "_(미기재)_")

    parts.append(_h(2, "필요 서류"))
    parts.append(_bullets(reqs) or "_(미기재)_")

    parts.append(_h(2, "주의/모호 사항"))
    parts.append(_bullets(cave) or "_(없음)_")

    parts.append(_h(3, "참고 근거(파일 p.xx)"))
    parts.append(_bullets(cites))
    return "\n\n".join(parts).strip() + "\n"

def print_surgery_md(data:dict, title_prefix="수술·질병 보장/청구 안내"):
    md = render_surgery_md(data, title_prefix=title_prefix)
    _show(md)
    return md

def save_surgery_md(data:dict, path:str="./export/surgery.md", title_prefix="수술·질병 보장/청구 안내") -> str:
    md = render_surgery_md(data, title_prefix=title_prefix)
    return _save_md(md, path)

print("✅ BLK7 OK | Markdown 포맷/저장 헬퍼 준비 완료")

✅ BLK7 OK | Markdown 포맷/저장 헬퍼 준비 완료


In [31]:
# === BLK8: Build single bundled Markdown (summary + grade + surgery) ===
from datetime import datetime
from pathlib import Path

# 안전 체크
try:
    summarize_policy_structured, grade_policy_structured, surgery_coverage_structured
    render_summary_md, render_grade_md, render_surgery_md, _save_md
    POLICY_PATH
except NameError as e:
    raise RuntimeError("필수 함수/변수가 없습니다. BLK1~BLK7 순서대로 실행했는지 확인하세요.") from e

# ---- 사용자 입력(필요 시 수정) ----
SURGERY_NAME = "관상동맥 우회술"   # 예: "CABG", "백내장수술" 등으로 변경 가능
PATIENT_AGE  = 45

# ---- 결과 생성 ----
sum_data   = summarize_policy_structured()
grade_data = grade_policy_structured()
surg_data  = surgery_coverage_structured(SURGERY_NAME, age=PATIENT_AGE)

# ---- 섹션 렌더링 (개별 MD 생성) ----
md_summary = render_summary_md(sum_data, title="1) 약관 핵심 요약")
md_grade   = render_grade_md(grade_data, title="2) 약관 품질 평가 (0~10점)")
md_surgery = render_surgery_md(surg_data, title_prefix=f"3) 수술·질병 보장/청구 안내 — {SURGERY_NAME}")

# ---- 헤더/목차/면책 문구 ----
ts     = datetime.now().strftime("%Y-%m-%d %H:%M")
fname  = Path(POLICY_PATH).name if 'POLICY_PATH' in globals() else "DBIS.pdf"

header = f"""# DBIS 약관 분석 합본

- **원본 파일:** {fname}
- **생성 시각:** {ts}

> ⚠️ 면책 고지: 본 문서는 정보 제공 목적이며 법률/의료/재정 자문이 아닙니다. 실제 보장/지급 여부와 금액은 보험사 심사 및 약관 원문, 개별 특약, 청구 서류 등에 따라 달라질 수 있습니다. 최종 판단은 반드시 보험사/설계사/전문가와 상담하세요.

## 목차
1. 약관 핵심 요약
2. 약관 품질 평가 (0~10점)
3. 수술·질병 보장/청구 안내 — {SURGERY_NAME} ({PATIENT_AGE}세)
"""

# ---- 합본 조립 ----
bundle_md = "\n\n".join([header.strip(), md_summary.strip(), md_grade.strip(), md_surgery.strip()]) + "\n"

# ---- 저장 ----
out_path = "./export/DBIS_bundle.md"
saved = _save_md(bundle_md, out_path)

print("✅ 합본 저장 완료 →", saved)
print("💡 노션/깃허브에 그대로 붙여넣을 수 있습니다.")

KeyboardInterrupt: 

In [ ]:
# === BLK9: Evidence Coverage Audit (premium/age, 3 diseases, exclusions/claims) ===
import re
import pandas as pd
from collections import defaultdict
from pathlib import Path

try:
    docs  # from previous blocks
except NameError:
    raise RuntimeError("docs 변수가 없습니다. 이전 블록을 먼저 실행하세요.")

# 1) 패턴/키워드
PREMIUM_TERMS = [r"보험료", r"월\s*보험료", r"연령", r"나이", r"세\b", r"가입\s*연령", r"납입", r"갱신", r"비갱신", r"보험료표"]
DISEASE_TERMS = sum([
    [r"암", r"악성신생물", r"cancer", r"malignant\s+neoplasm"],
    [r"뇌졸중", r"뇌졸증", r"중풍", r"stroke", r"cerebrovascular\s+accident", r"\bCVA\b"],
    [r"급성\s*심근경색", r"심근경색", r"acute\s+myocardial\s+infarction", r"\bAMI\b", r"myocardial\s+infarction"],
], [])
CLAIM_TERMS   = [r"청구", r"필요\s*서류", r"지급", r"면책", r"제외", r"대기\s*기간", r"보장하지\s*않는", r"제외\s*사항", r"예외"]

MONEY_PAT     = r"(\d{1,3}(?:[,\.]\d{3})+|\d+)\s*(원|만원|억|%)"
AGE_PAT       = r"(\d{1,3})\s*세"

def _count_hits(text:str, patterns) -> int:
    return sum(len(re.findall(p, text, flags=re.IGNORECASE)) for p in patterns)

def _cooccurrence_window(text:str, termsA, termsB, window=120):
    # termsA와 termsB가 근접(윈도우 내) 등장하는 횟수
    hits = 0
    for ta in termsA:
        for m in re.finditer(ta, text, flags=re.IGNORECASE):
            start = max(0, m.start()-window)
            end   = min(len(text), m.end()+window)
            seg = text[start:end]
            if any(re.search(tb, seg, flags=re.IGNORECASE) for tb in termsB):
                hits += 1
    return hits

rows = []
for d in docs:
    page = d.metadata.get("page", -1)
    txt = (d.page_content or "").strip()
    if not txt:
        continue
    # 기본 카운트
    prem_hits = _count_hits(txt, PREMIUM_TERMS)
    dis_hits  = _count_hits(txt, DISEASE_TERMS)
    clm_hits  = _count_hits(txt, CLAIM_TERMS)
    money_cnt = len(re.findall(MONEY_PAT, txt))
    age_cnt   = len(re.findall(AGE_PAT, txt))

    # 공존성(질병 용어 근처에 보장/지급/청구 등)
    cooc_benefit = _cooccurrence_window(txt, DISEASE_TERMS, [r"보장", r"진단비", r"수술비", r"지급"])
    cooc_claim   = _cooccurrence_window(txt, DISEASE_TERMS, [r"청구", r"필요\s*서류", r"면책", r"대기\s*기간"])

    rows.append({
        "page": page,
        "len": len(txt),
        "premium_hits": prem_hits,
        "disease_hits": dis_hits,
        "claim_hits": clm_hits,
        "money_tokens": money_cnt,
        "age_tokens": age_cnt,
        "cooc_benefit": cooc_benefit,
        "cooc_claim": cooc_claim,
    })

df = pd.DataFrame(rows).sort_values(["premium_hits","disease_hits","claim_hits","len"], ascending=False)
summary = {
    "pages_scanned": len(df),
    "pages_with_premium": int((df["premium_hits"]>0).sum()),
    "pages_with_disease": int((df["disease_hits"]>0).sum()),
    "pages_with_claim":   int((df["claim_hits"]>0).sum()),
    "pages_with_money":   int((df["money_tokens"]>0).sum()),
    "pages_with_age":     int((df["age_tokens"]>0).sum()),
    "cooc_benefit_total": int(df["cooc_benefit"].sum()),
    "cooc_claim_total":   int(df["cooc_claim"].sum()),
}
print("=== Evidence Coverage Summary ===")
for k,v in summary.items():
    print(f"{k}: {v}")

print("\n=== Top pages (by hits) ===")
display(df.head(10))

# 샘플 내용 미리보기
def show_page_snippet(page_no:int, n=600):
    matches = [d for d in docs if d.metadata.get("page")==page_no]
    if not matches:
        print("해당 페이지 없음:", page_no); return
    txt = (matches[0].page_content or "").strip().replace("\n"," ")
    print(f"[p.{page_no}] {txt[:n]}{'...' if len(txt)>n else ''}")

print("\n예시) show_page_snippet(0)  # 원하는 페이지 번호로 바꿔 확인")
